In [1]:
__author__ = "Tim Cuddeback"

import yfpy, json
from yfpy.query import YahooFantasySportsQuery
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = '12514' # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved
excel_file_name = 'Points.xlsx' # Name your file after you set up the headers

# Query yffpy to get data
yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

In [3]:
# Get standings data to get the points for the day
data = []
for i in range (0,17):
    d = yahoo_query.get_league_matchups_by_week(1)
    data.append(d)
# .get_league_teams(self)
# .get_league_players(self)
# .get_league_draft_results(self)
# .get_league_matchups_by_week(self,chosen_week)
# .get_team_info(self,team_id)
# .get_team_standings(self,team_id) # has current "streak" within
# .get_player_stats_by_week(self, player_key, chosen_week="current")

[2021-04-12 16:16:23,449 DEBUG] [yfpy.query.query.query] Response (JSON): {'fantasy_content': {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yahoo.com/f1', 'season': '2020', 'is_registration_over': 1, 'is_game_over': 1, 'is_offseason': 1}], 'time': '16.412973403931ms', 'copyright': 'Data provided by Yahoo! and STATS, LLC', 'refresh_rate': '60'}}
[2021-04-12 16:16:23,460 DEBUG] [yfpy.query.query.query] Data fetched with query URL: https://fantasysports.yahooapis.com/fantasy/v2/game/399/metadata?format=json
[2021-04-12 16:16:23,461 DEBUG] [yfpy.query.query.query] Response (Yahoo fantasy data extracted from: "fantasy_content"): {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yaho

In [4]:
data2 = []
df = pd.DataFrame()
for j in range(0,len(data)):
    for i in range(0,len(data[j])):
        data2.append(json.loads(str(data[j][i]['matchup'])))
    for i in range(0,len(data2)):
        row = {"week" : data2[i]['week'],
               "week_start" : data2[i]['week_start'],
               "team_id1" : data2[i]['teams'][0]['team']['managers']['manager']['manager_id'],
               "owner1" : data2[i]['teams'][0]['team']['managers']['manager']['nickname'],
               "team1" : data2[i]['teams'][0]['team']['name'],
               "points1" : data2[i]['teams'][0]['team']['team_points']['total'],
               "team_id2" : data2[i]['teams'][1]['team']['managers']['manager']['manager_id'],
               "owner2" : data2[i]['teams'][1]['team']['managers']['manager']['nickname'],
               "team2" : data2[i]['teams'][1]['team']['name'],
               "points2" : data2[i]['teams'][1]['team']['team_points']['total']}
        df1 = pd.DataFrame(row, index=[0])
        df = pd.concat([df1,df])